In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import numpy as np
np.set_printoptions(suppress=True)
import random
import math
from operator import itemgetter

In [2]:
df_sale = pd.read_csv('F_SALES_UN.csv',encoding='utf-16LE')
df_sale.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (4,5,47,58) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,SALES_ID,COUNTRY,BRAND_CODE,VIP_ID,VIP_CODE,ITEM_LOT_NUMBER,YEAR_MONTH,TXN_CNT,ITEM_CNT,FISCAL_MONTH,EX_RATE_ID,VAT_RATE,PRICERANGE_ID,STORE_CODE,SKU_CODE,LINE_NUMBER,DOCUMENT_NUMBER,ORDER_NUMBER,TRANSACTION_DATE,TRANSACTION_TYPE,TRANSACTION_TYPE_DESC,NET_AMOUNT,QUANTITY,ORIGINAL_SELLING_AMOUNT,RETAIL_SELLING_AMOUNT,COUPON_DISCOUNT_AMOUNT_ITEM,MEMO_COUPON_DISCOUNT_AMOUNT,ITEM_LINE_DISCOUNT_AMOUNT,TAX_AMOUNT,VIP_LINE_DISCOUNT_AMOUNT,SUM_OF_PROMOTION_AMOUNT,MEMO_OR_VIP_TOTAL_DISCOUNT,TRANSACTION_TIME,LAST_MODIFIED_TIME,TENDER,FISCAL_DATE,VAT_AMT,BI_NET_AMOUNT,FISCAL_WEEK,VIP_FIRST_TX,LAST_PURCHASE_TYPE,GENERIC_ARTICLE,MEMBER_SALES,ORIGINAL_SELLING_UNIT_PRICE,RETAIL_SELLING_UNIT_PRICE,MEMO_DISCOUNT,POST_DATE_TIME,SALES_TIER,PROMOTION_CODE_1,PROMOTION_DISCOUNT_AMOUNT_1,PROMOTION_CODE_2,PROMOTION_DISCOUNT_AMOUNT_2,PROMOTION_CODE_3,PROMOTION_DISCOUNT_AMOUNT_3,PROMOTION_CODE_4,PROMOTION_DISCOUNT_AMOUNT_4,PROMOTION_CODE_5,PROMOTION_DISCOUNT_AMOUNT_5,VIP_CENTER_CODE
0,66691614,China,TNF,0.0,NaN,191928377227,201901,1,1.0,201910,2,0.16,NaN,7013,NF0A2UC37GB1XXL,1,S21023311,7013_S21023311,2019-01-01,SA,Sales,1438.8,1,2398.0,2398.0,0.0,0.0,0.0,0.0,0.0,959.2,0.0,2019-01-01 15:52:39,2019-01-02 04:56:49.420000000,NaN,2019-01-01,198.4552,1438.8,201940,Not Assigned,NaN,NF0A2UC37GB1,Non-Member,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,66691615,China,TNF,17441109.0,CNVTNF341345118,191928378804,201901,0,1.0,201910,2,0.16,NaN,7024,NF0A365B7GC100L,4,S21024325,7024_S21024325,2019-01-01,SR,Sales Refund,-2798.0,-1,-3998.0,-3998.0,0.0,0.0,0.0,0.0,0.0,-1200.0,0.0,2019-01-01 10:50:23,2019-01-02 04:56:49.420000000,NaN,2019-01-01,-385.9310,-2798.0,201940,Repeat purchase,NaN,NF0A365B7GC1,Member,NaN,NaN,NaN,NaN,PN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CNVTNF
2,66691622,China,TNF,0.0,NaN,191931448136,201901,1,1.0,201910,2,0.16,NaN,7013,NF0A34N5MN8100S,1,S21023312,7013_S21023312,2019-01-01,SA,Sales,1888.6,1,2698.0,2698.0,0.0,0.0,0.0,0.0,0.0,809.4,0.0,2019-01-01 16:21:12,2019-01-02 04:56:49.420000000,NaN,2019-01-01,260.4966,1888.6,201940,Not Assigned,NaN,NF0A34N5MN81,Non-Member,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,66691623,China,TNF,17441109.0,CNVTNF341345118,191931585558,201901,1,1.0,201910,2,0.16,NaN,7024,NF0A2T5K5UG1070,2,S21024325,7024_S21024325,2019-01-01,SA,Sales,559.0,1,798.0,798.0,0.0,0.0,0.0,0.0,0.0,239.0,0.0,2019-01-01 10:50:23,2019-01-02 04:56:49.420000000,NaN,2019-01-01,77.1034,559.0,201940,Repeat purchase,NaN,NF0A2T5K5UG1,Member,NaN,NaN,NaN,NaN,PN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CNVTNF
4,66691630,China,TNF,0.0,NaN,191928574046,201901,1,1.0,201910,2,0.16,NaN,7024,NF0A32YI5QR1100,1,S21024324,7024_S21024324,2019-01-01,SA,Sales,710.0,1,1398.0,1398.0,0.0,0.0,0.0,0.0,0.0,688.0,0.0,2019-01-01 10:29:25,2019-01-02 04:56:49.420000000,NaN,2019-01-01,97.9310,710.0,201940,Not Assigned,NaN,NF0A32YI5QR1,Non-Member,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = df_sale[df_sale['VIP_ID']!=0]
df = df[df['VIP_ID'].notna()]
df = df.reset_index(drop = True)

In [4]:
#使用2021年数据进行测试，数据量57w
#df['YEAR_MONTH'] = df['YEAR_MONTH'].astype(str)
#df_test = df[df['YEAR_MONTH'].str.contains('2021')]
#df_test = df_test.reset_index(drop = True)
#df_test = df

In [5]:
#将user-item数据拆成list，算法输入格式为list
def ori_list_prepare(df):
    df_list = []
    for i in range(len(df)):
        df_list.append([df.loc[i,'VIP_ID'],df.loc[i,'GENERIC_ARTICLE']])
    return df_list

In [6]:
#train-test split code，这里M为需要分为几部分，K是一个小于M的整数
def Splitdata(data, M, k, seed):
    test = dict()
    train = dict()
    random.seed(seed)
    for user, item in data:
        rdm = random.randint(0, M)
        if rdm == k:
            if user not in test:
                test[user] = set()
            test[user].add(item)
        else:
            if user not in train:
                train[user] = set()
            train[user].add(item)
    return train, test

In [7]:
#ITEM sim
def ItemSimilarity(train):
    C = dict()
    N = dict()
    for u, items in train.items():
        for i in items:
            if i not in N:
                N[i] = 0
            N[i] += 1
            for j in items:
                if i == j:
                    continue
                if i not in C:
                    C[i] = dict()
                if j not in C[i]:
                    val = 1 / math.log(1 + len(items)*1.0)
                    C[i].update({j: val})
                else:
                    val = C[i][j] + 1 / math.log(1 + len(items)*1.0)
                    C[i].update({j: val})
    W = dict()
    for i, related_items in C.items():
        for j, cij in related_items.items():
            if i not in W:
                W[i] = dict()
 
            val = cij / math.sqrt(N[i] * N[j])
            W[i].update({j: val})
 
    return W

In [8]:
#ITEM 推荐系统
def ItemCFRecommend(user_id,train, W, K):
    rank = dict()
    ru = train[user_id]
    for i in ru:
        for j, wj in sorted(W[i].items(), key=itemgetter(1), reverse=True)[0:K]:
            if j in ru:
                continue
            if j not in rank:
                rank[j] = wj
            else:
                rank[j] +=  wj
    return rank

In [9]:
#准备train、test
df_full = ori_list_prepare(df)
df_final = dict()

for user, item in df_full:
    if user not in df_final:
        df_final[user] = set()
    df_final[user].add(item)

In [10]:
W = ItemSimilarity(df_final)

In [24]:
#rank = ItemCFRecommend(17441109.0,train,W,80)
#rank

{'NF0A496SCZ6100L': 0.06452351755126096,
 'NF0A3RD3TJ21070': 0.019830025817321205,
 'NF0A3LDZHCH1036REG': 0.015145432390133827,
 'NF0A496S70M10XL': 0.04741228445676587,
 'NF0A3LDZJK31034REG': 0.013116329201155564,
 'NF0A3VN6ZCE10XL': 0.011112129081850351,
 'NF0A496T70M10XL': 0.01580758214298134,
 'NF0A496RCZ610XL': 0.0211144932759909,
 'NF0A496UCZ610XL': 0.02310171848996711,
 'NF0A496R70M10XL': 0.06926321460158848,
 'NF0A5A4YZBJ10XL': 0.03938968875518323,
 'NF0A4ALAJK31008REG': 0.03938968875518323,
 'NF0A3CJ4EFS10XL': 0.008765365627777088,
 'NF0A3V2N7D610XL': 0.008373009522280258,
 'NF0A496SCZ61XXL': 0.008220177619287965,
 'NF0A3WZZBQW1110': 0.014086307276561447,
 'NF0A3S8DMN81LXL038': 0.00792054139262086,
 'NF0A496UCZ6100L': 0.028769133665950797,
 'NF0A4N9USF7100L': 0.017579175425716524,
 'NF0A4U8MV381002REG': 0.024396493079641347,
 'NF0A496S68210XL': 0.007634722902580408,
 'NF0A3C8DNL4100L': 0.0074587089978144115,
 'NF0A3RKGDYZ1XXL': 0.00743911200141283,
 'NF0A3NOJCZ6100M': 0.0074271

In [19]:
#list(rank.keys())[:3]

['NF0A4NAGJK3100L', 'NF0A4NAGJK31XXL', 'NF0A4NAGR13100L']

In [11]:
user_key = list(df_final.keys())
result = pd.DataFrame()
for i in user_key:
    try:
        rank = ItemCFRecommend(i,df_final,W,80)
        result = result.append({'USER-ID':i,'购买记录':list(df_final[i]),'Top10-SKU':list(rank.keys())[:10]},ignore_index = True)
    except:
        pass
        continue


In [14]:
result.head(200)

,USER-ID,购买记录,Top10-SKU
0,17441109.0,"[NF0A3VT1JK31, NF0A4NCRAVM1, NF0A496R70M1, NF0...","[NF0A3VT1HBM1, NF0A3L8OKX71, NF0A39IBGSQ1, NF0..."
1,17516129.0,"[NF0A3VSFJK31, NF0A4NAGR131, NF0A3VTHJK31, NF0...","[NF0A3FI71SB1, NF0A49B6JK31, NF0A366JJK31, NF0..."
2,17525617.0,"[NF0A4UC6JK31, NF0A39IA5VF1, NF0A5JVNH7E1, NF0...","[NF0A4UC6FN41, NF0A4UC66821, NF0A4UC5JK31, NF0..."
3,17586086.0,"[NF0A4NCFFN41, NF0A3L8YJK31, NF0A496ZZBJ1, NF0...","[NF0A4NCAJK31, NF0A4NCFHK41, NF0A499OFN41, NF0..."
4,17488492.0,"[NF0A4UC5JK31, NF0A3V8RDYX1, NF0A2SAEJA31, NF0...","[NF0A4UC5FN41, NF0A4UC56821, NF0A4UC6JK31, NF0..."
5,17566456.0,"[NF0A3RDFKY41, NF0A4NB8RG11, NF0A4U8ZJK31, NF0...","[NF0A3RDEAUN1, NF0A3RDD5PZ1, NF0A39ISAJ01, NF0..."
6,17475115.0,"[NF0A3KY6PJ81, NF0A4NCFFN41, NF0A3CNPKG71, NF0...","[NF0A3KY6KS71, NF0A499KZCF1, NF0A3M1VJY81, NF0..."
7,17451612.0,[NF0A3KTCKX71],"[NF0A3IFLJK31, NF0A2VCRKX71, NF0A3V4OKS71, NF0..."
8,17476158.0,"[NF0A3VT1JK31, NF0A4NCRAVM1, NF0A4PFOZCN1, NF0...","[NF0A3VT1HBM1, NF0A3L8OKX71, NF0A39IBGSQ1, NF0..."
9,17552213.0,"[NF0A4NCRAVM1, NF00CNK4H2G1, NF0A46BWNEC1, NF0...","[NF0A4NCRDYX1, NF0A4NCRKS71, NF0A46KXDYZ1, NF0..."


In [13]:
len(result)

1198144

In [15]:
result.to_csv('Item-Item_RECOMMEND_result(spring4).csv',index = False,encoding="utf_8_sig")

# SPU

In [13]:
#SPU_list = list(W.keys())

In [14]:
#len(SPU_list)

28996

In [17]:
#test_list = sorted(W['NF0A5B3EK2K10XL'].items(), key=itemgetter(1), reverse=True)[0:5]
#test_list

[('NF0A5B3EK2K1XXL', 0.037225484459782014),
 ('NF0A5B3EK2K100L', 0.033387222492203016),
 ('NF0A5B3EZCF10XL', 0.024508393744217226),
 ('NF0A5B3E37X10XL', 0.024014905106680823),
 ('NF0A4UCGWZY10XL', 0.019137786389141456)]

In [18]:
#temp = []
#for key,value in test_list:
    #temp.append(key)
#temp

['NF0A5B3EK2K1XXL',
 'NF0A5B3EK2K100L',
 'NF0A5B3EZCF10XL',
 'NF0A5B3E37X10XL',
 'NF0A4UCGWZY10XL']

In [29]:
#result = pd.DataFrame()
#for i in SPU_list:
    #try:
        #sim_list = sorted(W[i].items(), key=itemgetter(1), reverse=True)[0:10]
        #result = result.append({'商品SKU':i,'相关性前十商品SKU':sim_list},ignore_index = True)
    #except:
        #pass
        #continue

In [30]:
#result.head()

,商品SKU,相关性前十商品SKU
0,NF0A46LBJK313XL,"[(NF0A536GFN413XL, 0.04497383593415881), (NF0A..."
1,NF0A4NFBKX7100L,"[(NF0A4NFBKX7100M, 0.04262189188999447), (NF0A..."
2,NF0A2T5K5UG1060,"[(NF0A3YUXWU51080, 0.08340647828484926), (NF0A..."
3,NF0A3VSKJK3100M,"[(NF0A3VSKJK3100S, 0.05478412148205356), (NF0A..."
4,NF0A4NCRKS710XL,"[(NF0A4NCRAVM10XL, 0.04688950627250225), (NF0A..."


In [27]:
#list1 = result.iloc[0,1]

In [28]:
#list1

[('NF0A536GFN413XL', 0.04497383593415881),
 ('NF0A3YUPKZ21115', 0.0411025896287918),
 ('NF0A46LBDYY13XL', 0.036346484870258954),
 ('NF0A46LBJK31XXL', 0.028568042436123337),
 ('NF0A47H3ZU51130', 0.02844201499036111),
 ('NF0A5B41NL413XL', 0.020354553397434163),
 ('NF0A4ALAJK31008REG', 0.017787218850271288),
 ('NF0A5A4YZBJ10XL', 0.017787218850271288),
 ('NF0A3CNP8UX100S', 0.01699606125091043),
 ('NF0A537SX71100S', 0.01690601426995966)]

In [31]:
#result.to_csv('SKU-相关.csv',index = False,encoding="utf_8_sig")

# Acuracy

In [65]:
#rank = ItemCFRecommend(17441109.0, train, W, 80)
#rk = sorted(rank.items(), key=itemgetter(1), reverse=True)[0:5]
#for item, pui in rk:
    #print(item,pui)

NF0A46LBJK31XXL 0.05748557125107071
NF0A4NAXJK31008REG 0.054685214398945456
NF0A4CETZU31100 0.05376606970382601
NF0A3V1HKX71070 0.053090512437617636
NF0A4NB5R131XXL 0.05077757298972062


In [17]:
count = 0
for i in range(len(result)):
    if result.iloc[i,1] < result.iloc[i,2]:
        count = count + 1
print(count/len(result))

0.5057622456065382
